## Preamble

### Template Utils

In [ ]:
%load_ext autoreload

In [ ]:
import os as _os

_os.chdir(_os.environ["PROJECT_ROOT"])
_os.path.realpath(_os.path.curdir)

### Imports

In [ ]:
import os
import subprocess
import time
from itertools import chain, product
from tempfile import mkstemp

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import sfacts as sf
import statsmodels as sm
import statsmodels.formula.api as smf
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

import lib.plot
import lib.thisproject.data
from lib.pandas_util import align_indexes, aligned_index, idxwhere, invert_mapping

### Set Style

In [ ]:
sns.set_context("talk")
plt.rcParams["figure.dpi"] = 50

### Papermill parameters

In [ ]:
# This cell is tagged "parameters" for papermill.
# See <https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work> for some gotchas.
# NOTE: *ALL* parameters should be passed to papermill. Values set here are only for prototyping.
species_id = "102327"

species_taxonomy_inpath = "ref/gtpro/species_taxonomy_ext.tsv"
sample_to_spgc_inpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.spgc_ss-all.strain_samples.tsv"
sfacts_fit_inpath = (
    f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.world.nc"
)
# ref_geno_inpath = f"data/species/sp-{species_id}/midasdb.geno.nc"  # "De-novo GT-Pro genotype"
ref_geno_inpath = (
    f"data/species/sp-{species_id}/gtpro_ref.mgtp.nc"  # "Reference GT-Pro genotype"
)
spgc_meta_inpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.strain_meta.tsv"
ref_gene_copy_number_uhgg_inpath = (
    f"ref/midasdb_uhgg_pangenomes/{species_id}/gene75.reference_copy_number.nc"
)
spgc_gene_uhgg_inpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.strain_gene.tsv"
spgc_gene_uhgg_depth_inpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30.strain_depth_ratio.tsv"
spgc_gene_uhgg_corr_inpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30.strain_correlation.tsv"
uhgg_x_eggnog_inpath = (
    f"data/species/sp-{species_id}/pangenome.centroids.emapper.gene_x_eggnog.tsv"
)
uhgg_x_top_eggnog_inpath = (
    f"data/species/sp-{species_id}/pangenome.centroids.emapper.gene_x_top_eggnog.tsv"
)
uhgg_gene_length_inpath = f"ref/midasdb_uhgg/pangenomes/{species_id}/cluster_info.txt"
gene_annotations_inpath = f"data/species/sp-{species_id}/pangenome.centroids.emapper.d/proteins.emapper.annotations"
uhgg_depth_inpath = (
    f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gene99-v22-agg75.depth2.nc"
)
species_depth_inpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gene99-v22-agg75.spgc_specgene-ref-t25-p95.species_depth.tsv"
mgen_inpath = "meta/hmp2/mgen.tsv"
preparation_inpath = "meta/hmp2/preparation.tsv"
stool_inpath = "meta/hmp2/stool.tsv"
subject_inpath = "meta/hmp2/subject.tsv"

# TODO: Export SPGC gene info: prevalence, passes filtering against references,
#    concordance with SNP dissimilarity in each genome set (both using SPGC-only, SPGC+refs, refs-only),
#    cluster membership for each clustering analysis (SPGC-only, SPGC+refs, refs-only),
#    compiled metadata (length, COG category table), etc.
gene_stats_outpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.gene_stats.tsv"
# TODO: Export info for each SPGC strain: passes filtering? relationship between
#   minimum genotype distance to a reference and the associated minimum gene
#   distance (both weighted and unweighted)
spgc_strain_stats_outpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.spgc_strain_stats.tsv"
# TODO: Export info for each reference strain: relationship between
#   minimum genotype distance to a reference and the associated minimum gene
#   distance (both weighted and unweighted)
ref_strain_stats_outpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.ref_strain_stats.tsv"

# TODO: Export anything else I need for cross-species comparisons. What might that be?
# strain_mwas_outpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.strain_mwas.tsv"
html_outpath = f"data/group/xjin_hmp2/species/sp-{species_id}/r.proc.gtpro.sfacts-fit.gene99-v22-agg75.spgc_specgene-ref-t25-p95_ss-all_t-30_thresh-corr100-depth250.spgc_ref_comparison.html"

## Data Loading / Validation

In [ ]:
!date

#### Taxonomy

In [ ]:
species_taxonomy = lib.thisproject.data.load_species_taxonomy(species_taxonomy_inpath)
species_taxonomy.loc[species_id]

### Gene Annotations

In [ ]:
uhgg_x_eggnog = pd.read_table(uhgg_x_eggnog_inpath)

In [ ]:
uhgg_x_top_eggnog = pd.read_table(uhgg_x_top_eggnog_inpath)

In [ ]:
uhgg_gene_length = (
    pd.read_table(uhgg_gene_length_inpath)
    .groupby("centroid_75")
    .centroid_99_length.mean()
)

In [ ]:
eggnog_column_names = "query seed_ortholog evalue score eggNOG_OGs max_annot_lvl COG_category Description Preferred_name GOs EC KEGG_ko KEGG_inpathway KEGG_Module KEGG_Reaction KEGG_rclass BRITE KEGG_TC CAZy BiGG_Reaction PFAMs".split(
    " "
)
_gene_annotations = (
    pd.read_table(
        gene_annotations_inpath,
        comment="#",
        names=eggnog_column_names,
        index_col="query",
    )
    .rename_axis(index="gene_id")
    .replace({"-": np.nan})
)
_gene_annotations = uhgg_gene_length.to_frame().join(_gene_annotations)

_gene_annotations.info()

In [ ]:
gene_x_cog_category1 = (
    _gene_annotations.COG_category.fillna("-").apply(list).explode()[lambda x: x != "-"]
)
gene_x_cog_category1

In [ ]:
cog_x_category = pd.read_table(
    "ref/cog-20.meta.tsv",
    names=["cog", "cog_category", "description", "short_name", "_4", "_5", "_6"],
    index_col="cog",
).cog_category
cog_x_category

In [ ]:
_gene_annotations.eggNOG_OGs

In [ ]:
gene_x_cog = (
    _gene_annotations.eggNOG_OGs.fillna("")
    .str.split(",")
    .explode()[lambda x: x.str.startswith("COG")]
    .str.split("@")
    .str[0]
)
gene_x_cog.value_counts().head()
gene_x_cog_category2 = gene_x_cog.map(cog_x_category).dropna().apply(list).explode()
gene_x_cog_category2

In [ ]:
gene_x_cog_category = (
    pd.concat(
        [
            gene_x_cog_category1,
            gene_x_cog_category2,  # FIXME: Which metadata table do I want?
        ]
    )
    .reset_index()
    .drop_duplicates()
)

gene_x_cog_category.columns = ["centroid_75", "cog_category"]
gene_x_cog_category = gene_x_cog_category.set_index("centroid_75").cog_category
gene_x_cog_category.shape[0]

In [ ]:
gene_annotations = _gene_annotations.assign(
    COG_category=gene_x_cog_category.sort_values()
    .reset_index()
    .groupby("centroid_75")
    .apply(lambda x: "".join(x.cog_category.values))
).assign(COG_category=lambda x: x.COG_category.fillna(""))

In [ ]:
gene_x_cog_category_matrix = (
    gene_x_cog_category.reset_index()
    .assign(tally=True)
    .set_index(["centroid_75", "cog_category"])
    .tally.unstack("cog_category")
    .fillna(False)
    .reindex(gene_annotations.index, fill_value=False)
    .assign(no_category=lambda x: x.sum(1) == 0)
)
gene_x_cog_category_matrix

### Raw Data

In [ ]:
species_depth = pd.read_table(
    species_depth_inpath, names=["sample", "depth"], index_col=["sample"]
).depth
species_depth

In [ ]:
mgen = pd.read_table(mgen_inpath, index_col="library_id")
preparation = pd.read_table(preparation_inpath, index_col="preparation_id")
stool = pd.read_table(stool_inpath, index_col="stool_id")
subject = pd.read_table(subject_inpath, index_col="subject_id")

mgen_meta = (
    mgen.join(preparation, on="preparation_id", lsuffix="_mgen", rsuffix="_preparation")
    .join(stool, on="stool_id")
    .join(subject, on="subject_id")
)

In [ ]:
uhgg_depth = xr.load_dataarray(uhgg_depth_inpath)
subject_uhgg_depth = (
    mgen_meta[["subject_id"]]
    .join(uhgg_depth.to_pandas())
    .groupby("subject_id")
    .mean()
    .dropna()
)

In [ ]:
uhgg_depth_ratio = uhgg_depth / species_depth.to_xarray().sel(sample=uhgg_depth.sample)

In [ ]:
subject_mean_species_depth = (
    mgen_meta[["subject_id"]]
    .join(species_depth)
    .groupby("subject_id")
    .depth.mean()
    .dropna()
)

In [ ]:
total_subject_uhgg_depth = (
    mgen_meta[["subject_id"]]
    .join(uhgg_depth.to_pandas())
    .groupby("subject_id")
    .sum()
    .dropna()
)
total_subject_species_depth = (
    mgen_meta[["subject_id"]]
    .join(species_depth)
    .groupby("subject_id")
    .depth.sum()
    .dropna()
)
subject_uhgg_depth_ratio = total_subject_uhgg_depth.divide(
    total_subject_species_depth, axis=0
)

### SPGC Strains

In [ ]:
sample_to_spgc = pd.read_table(sample_to_spgc_inpath, index_col="sample").strain.astype(
    str
)

spgc_palette = lib.plot.construct_ordered_palette(
    sample_to_spgc, other=(0.8, 0.8, 0.8, 1.0)
)

In [ ]:
sfacts_fit = sf.World.load(sfacts_fit_inpath).drop_low_abundance_strains(0.5)
# Strains should be str not int.
sfacts_fit.data["strain"] = sfacts_fit.strain.values.astype(str)
print(dict(sfacts_fit.sizes))

spgc_est_mgtp = sf.Metagenotype(
    sfacts_fit.metagenotype.data.sel(sample=sample_to_spgc.index)
    .groupby(sample_to_spgc.to_xarray())
    .sum()
    .rename(strain="sample")
)
spgc_est_geno = spgc_est_mgtp.to_estimated_genotype(pseudo=0)

# Pre-calculate shared heatmap decorations
position_ss = sfacts_fit.random_sample(
    position=min(500, sfacts_fit.sizes["position"])
).position
w = sfacts_fit.sel(position=position_ss)
sample_linkage = w.unifrac_linkage()
try:
    position_linkage = spgc_est_geno.linkage("position")
except ValueError as err:
    print(err)
    position_linkage = None
sample_colors = w.sample.to_series().map(sample_to_spgc).map(spgc_palette)
spgc_linkage = w.genotype.linkage()
spgc_colors = w.strain.to_series().map(spgc_palette)

In [ ]:
sf.plot.plot_community(sfacts_fit.sel(position=position_ss))

In [ ]:
sf.plot.plot_metagenotype(
    sfacts_fit.sel(position=position_ss),
    col_linkage_func=lambda w: w.community.linkage("sample"),
)

In [ ]:
g = sf.data.Genotype.concat(
    dict(mgen=spgc_est_geno, fit=sfacts_fit.genotype), dim="strain"
)
g_pdist = g.pdist()
g_linkage = g.linkage()
sf.plot.plot_genotype(
    g.sel(position=position_ss), transpose=True, col_linkage=g_linkage
)

### Ref Strains

In [ ]:
# "Reference GT-Pro genotype"
ref_geno = sf.Metagenotype.load(ref_geno_inpath).to_estimated_genotype()
ref_geno.data["strain"] = ref_geno.strain.to_series().map(
    lambda s: "UHGG" + s[len("GUT_GENOME") :]
)
ref_geno.shape

In [ ]:
reference_meta = (
    pd.read_table("ref/uhgg_genomes_all_4644.tsv", index_col="Genome")
    .rename_axis(index="genome_id")[
        lambda x: x.MGnify_accession == "MGYG-HGUT-" + species_id[1:]
    ]
    .rename(lambda s: "UHGG" + s[len("GUT_GENOME") :])
    .loc[ref_geno.strain]
)

In [ ]:
bins = np.linspace(0, 100, num=101)
plt.hist(reference_meta.Completeness, bins=bins)
plt.hist(reference_meta.Contamination, bins=bins)
None

In [ ]:
# NOTE: Select any ref genotype that is within the top-10 closest distances from an SPGC strain.
spgc_to_ref_geno_cdist = ref_geno.sel(
    position=spgc_est_geno.position,
    strain=idxwhere(
        (reference_meta.Completeness > 97) & (reference_meta.Contamination < 2)
    ),
).cdist(spgc_est_geno)
ref_list = list(
    spgc_to_ref_geno_cdist.apply(lambda x: x.sort_values().head(10).index)
    .stack()
    .unique()
)

In [ ]:
spgc_meta = pd.read_table(spgc_meta_inpath, index_col="strain").rename(str)
print(spgc_meta.shape)
spgc_meta

In [ ]:
ref_gene_copy_number_uhgg = xr.load_dataarray(ref_gene_copy_number_uhgg_inpath)
ref_gene_uhgg = (ref_gene_copy_number_uhgg > 0).astype(int).to_pandas().T

In [ ]:
ref_gene_uhgg.shape

In [ ]:
spgc_gene_uhgg = pd.read_table(spgc_gene_uhgg_inpath, index_col="gene_id").rename_axis(
    columns="strain"
)

In [ ]:
ref_num_genes_uhgg = ref_gene_uhgg.sum()
spgc_num_genes_uhgg = spgc_gene_uhgg.sum()

In [ ]:
_ref_gene_uhgg_prevalence = ref_gene_uhgg[ref_list].mean(1)
ref_gene_uhgg_entropy = (
    -_ref_gene_uhgg_prevalence * np.log2(_ref_gene_uhgg_prevalence)
).fillna(0)
plt.hist(ref_gene_uhgg_entropy)

### Strain Selection / Filtering

In [ ]:
x = spgc_meta[lambda x: x.species_gene_frac > 0.9].num_genes
y = ref_num_genes_uhgg
_df, _loc, _scale = sp.stats.t.fit(x.values, fix_df=2)
_dist0 = sp.stats.t(_df, _loc, _scale)
_dist1 = sp.stats.norm(_loc, _scale)

thresh_max_num_uhgg_genes = _dist1.ppf(0.999)
thresh_min_num_uhgg_genes = _dist1.ppf(0.001)


bins = np.linspace(0, x.max() * 1.5, num=50)
xx = np.linspace(0, x.max() * 1.5, num=1000)

plt.hist(x, density=True, bins=bins, alpha=0.2)
plt.hist(y, density=True, bins=bins, alpha=0.2)

plt.plot(xx, _dist0.pdf(xx), color="k")
plt.plot(xx, _dist1.pdf(xx), color="k", linestyle="--")
plt.axvline(thresh_max_num_uhgg_genes, lw=1, linestyle="--", color="k")
plt.axvline(thresh_min_num_uhgg_genes, lw=1, linestyle="--", color="k")

In [ ]:
thresh_min_num_uhgg_genes, thresh_max_num_uhgg_genes

In [ ]:
plt.scatter(
    "sum_depth",
    "species_gene_frac",
    c=(spgc_meta.num_genes - _loc) / _scale,
    data=spgc_meta,
    norm=mpl.colors.PowerNorm(1 / 1, vmin=-4, vmax=4),
)
plt.xscale("log")
plt.yscale("logit")
plt.axvline(1, lw=1, color="k", linestyle="--")
plt.axhline(0.9, lw=1, color="k", linestyle="--")
plt.colorbar()

In [ ]:
# NOTE: Select SPGC strains that pass various filters
spgc_list = idxwhere(
    (spgc_meta.sum_depth > 1)
    & (spgc_meta.species_gene_frac > 0.9)
    & (spgc_num_genes_uhgg <= thresh_max_num_uhgg_genes)
    & (spgc_num_genes_uhgg >= thresh_min_num_uhgg_genes)
)

print(len(ref_list), len(spgc_list))

In [ ]:
assert len(spgc_list) > 2

In [ ]:
strain_geno = sf.Genotype.concat(
    dict(
        ref=ref_geno.sel(strain=ref_list, position=spgc_est_geno.position),
        spgc=spgc_est_geno.sel(strain=spgc_list),
    ),
    dim="strain",
    rename=False,
).mlift("fillna", 0.5)

strain_geno_pdist = strain_geno.pdist(q=1)
strain_geno_linkage = strain_geno.linkage(pdist_kwargs=dict(q=1), optimal_ordering=True)

In [ ]:
genome_type = reference_meta.Genome_type.reindex(strain_geno.strain)
genome_type.loc[genome_type.index.isin(spgc_list)] = "SPGC"
genome_type_order = ["Isolate", "MAG", "SPGC"]
genome_type_palette = lib.plot.construct_ordered_palette(genome_type_order)

In [ ]:
isolate_list = idxwhere(genome_type == "Isolate")
mag_list = idxwhere(genome_type == "MAG")

In [ ]:
fig, ax = plt.subplots(figsize=(2, 1))
for _genome_type in genome_type_order:
    ax.scatter([], [], color=genome_type_palette[_genome_type], label=_genome_type)
ax.legend()

lib.plot.hide_axes_and_spines(ax)

In [ ]:
fig, ax = plt.subplots(figsize=(2, 1))
for _genome_type in genome_type_order:
    ax.scatter([], [], color=genome_type_palette[_genome_type], label=_genome_type)
ax.legend(ncols=3)

lib.plot.hide_axes_and_spines(ax)

In [ ]:
_colors = genome_type.map(genome_type_palette)
sf.plot.plot_genotype(
    strain_geno.sel(position=position_ss),
    transpose=True,
    col_linkage=strain_geno_linkage,
    col_colors=_colors,
)

In [ ]:
_colors = genome_type.map(genome_type_palette)
sns.clustermap(
    strain_geno_pdist,
    row_colors=_colors,
    col_colors=_colors,
    row_linkage=strain_geno_linkage,
    col_linkage=strain_geno_linkage,
    vmin=0,
    vmax=1,
    xticklabels=0,
    yticklabels=0,
    # figsize=(40, 40),
)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2))
sns.heatmap([[]], vmin=0, vmax=1, ax=ax, cbar_kws=dict(location="bottom"))
lib.plot.hide_axes_and_spines(ax=ax)

In [ ]:
from scipy.spatial.distance import squareform

bins = np.linspace(0, 1)
plt.hist(
    squareform(strain_geno_pdist.loc[ref_list, ref_list]),
    bins=bins,
    histtype="step",
    density=True,
    label="ref-to-ref",
)
plt.hist(
    strain_geno_pdist.loc[ref_list, spgc_list].values.flatten(),
    bins=bins,
    histtype="step",
    density=True,
    label="spgc-to-ref",
)
plt.hist(
    squareform(strain_geno_pdist.loc[spgc_list, spgc_list]),
    bins=bins,
    histtype="step",
    density=True,
    label="spgc-to-spgc",
)
plt.legend()

In [ ]:
bins = np.linspace(0, 1, num=200)

_pdist = strain_geno_pdist + np.eye(len(strain_geno_pdist))

plt.hist(
    _pdist.loc[ref_list, ref_list].min(),
    bins=bins,
    histtype="step",
    cumulative=True,
    density=True,
    label="ref-to-ref",
)
plt.hist(
    _pdist.loc[ref_list, spgc_list].min(),
    bins=bins,
    histtype="step",
    cumulative=True,
    density=True,
    label="spgc-to-ref",
)
plt.hist(
    _pdist.loc[spgc_list, ref_list].min(),
    bins=bins,
    histtype="step",
    cumulative=True,
    density=True,
    label="ref-to-spgc",
)
plt.hist(
    _pdist.loc[spgc_list, spgc_list].min(),
    bins=bins,
    histtype="step",
    cumulative=True,
    density=True,
    label="spgc-to-spgc",
)

plt.legend()
# plt.axvline(0.43, lw=1, linestyle='--', color='k')
plt.xlabel("minimum distance")
plt.ylabel("cumulative fraction")

In [ ]:
spgc_gene_uhgg.shape, ref_gene_uhgg.shape

In [ ]:
strain_gene_uhgg = pd.concat(
    [ref_gene_uhgg[ref_list], spgc_gene_uhgg[spgc_list]], axis=1
).fillna(0)[lambda x: x.sum(1) > 0]
strain_gene_uhgg.shape

In [ ]:
bins = np.linspace(0, 10000)
plt.hist(
    strain_gene_uhgg[ref_list].sum(),
    bins=bins,
    histtype="step",
    label="ref",
    density=True,
)
plt.hist(
    strain_gene_uhgg[spgc_list].sum(),
    bins=bins,
    histtype="step",
    label="spgc",
    density=True,
)
plt.legend()

None

## Strain Geno/Gene Spaces

In [ ]:
# NOTE: This will take ~2 minutes to run for 40,000 genes.
gene_uhgg_cdmat = sp.spatial.distance.pdist(strain_gene_uhgg, metric="cosine")
gene_uhgg_pdist = pd.DataFrame(
    sp.spatial.distance.squareform(gene_uhgg_cdmat),
    index=strain_gene_uhgg.index,
    columns=strain_gene_uhgg.index,
)

In [ ]:
gene_uhgg_linkage = sp.cluster.hierarchy.linkage(gene_uhgg_cdmat, method="average")
gene_uhgg_linkage.shape

In [ ]:
strain_gene_uhgg_cdmat = sp.spatial.distance.pdist(
    strain_gene_uhgg.T,
    metric="cosine",
    w=ref_gene_uhgg_entropy.loc[strain_gene_uhgg.index],
)
strain_gene_uhgg_pdist = pd.DataFrame(
    sp.spatial.distance.squareform(strain_gene_uhgg_cdmat),
    index=strain_gene_uhgg.columns,
    columns=strain_gene_uhgg.columns,
)

assert (strain_geno_pdist.index == strain_gene_uhgg_pdist.index).all()

In [ ]:
strain_gene_uhgg_linkage = sp.cluster.hierarchy.linkage(
    strain_gene_uhgg_cdmat, method="average", optimal_ordering=True
)

In [ ]:
x = strain_gene_uhgg
_col_linkage = strain_geno_linkage
_row_linkage = gene_uhgg_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)

sns.clustermap(x, row_cluster=False, col_linkage=_col_linkage, col_colors=_col_colors)

In [ ]:
x = strain_gene_uhgg
_col_linkage = strain_gene_uhgg_linkage
_row_linkage = gene_uhgg_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)


sns.clustermap(x, row_cluster=False, col_linkage=_col_linkage, col_colors=_col_colors)

In [ ]:
spgc_gene_uhgg_depth = (
    pd.read_table(spgc_gene_uhgg_depth_inpath, index_col=["gene_id", "strain"])
    .depth.unstack("strain")
    .rename(columns=str)
)
spgc_gene_uhgg_corr = (
    pd.read_table(spgc_gene_uhgg_corr_inpath, index_col=["gene_id", "strain"])
    .correlation.unstack("strain")
    .rename(columns=str)
)

In [ ]:
_ref = ref_gene_uhgg[ref_list]
_spgc = spgc_gene_uhgg_depth[spgc_list]
strain_gene_uhgg_depth = (
    pd.concat([_ref, _spgc], axis=1).fillna(0).loc[strain_gene_uhgg.index]
)

In [ ]:
_ref = ref_gene_uhgg[ref_list]
_spgc = spgc_gene_uhgg_corr[spgc_list]
strain_gene_uhgg_corr = (
    pd.concat([_ref, _spgc], axis=1).fillna(0).loc[strain_gene_uhgg.index]
)

In [ ]:
x = strain_gene_uhgg_depth
_col_linkage = strain_gene_uhgg_linkage
_row_linkage = gene_uhgg_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)


sns.clustermap(
    x,
    row_cluster=False,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    norm=mpl.colors.PowerNorm(1 / 2, vmin=0, vmax=2),
)

In [ ]:
x = strain_gene_uhgg_corr
_col_linkage = strain_gene_uhgg_linkage
_row_linkage = gene_uhgg_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)

sns.clustermap(x, row_cluster=False, col_linkage=_col_linkage, col_colors=_col_colors)

In [ ]:
_pdistA = strain_geno_pdist
_pdistB = strain_gene_uhgg_pdist

plt.scatter(
    squareform(_pdistA.loc[ref_list, ref_list]),
    squareform(_pdistB.loc[ref_list, ref_list]),
    s=1,
    alpha=0.5,
    label="ref-ref",
)
print(
    "ref-ref",
    sp.stats.pearsonr(
        squareform(_pdistA.loc[ref_list, ref_list]),
        squareform(_pdistB.loc[ref_list, ref_list]),
    ),
)

plt.scatter(
    squareform(_pdistA.loc[spgc_list, spgc_list]),
    squareform(_pdistB.loc[spgc_list, spgc_list]),
    s=1,
    alpha=0.5,
    label="spgc-spgc",
)
print(
    "spgc-spgc",
    sp.stats.pearsonr(
        squareform(_pdistA.loc[spgc_list, spgc_list]),
        squareform(_pdistB.loc[spgc_list, spgc_list]),
    ),
)

plt.scatter(
    _pdistA.loc[spgc_list, ref_list].values.flatten(),
    _pdistB.loc[spgc_list, ref_list].values.flatten(),
    s=1,
    alpha=0.5,
    label="spgc-ref",
)
print(
    "spgc-ref",
    sp.stats.pearsonr(
        _pdistA.loc[spgc_list, ref_list].values.flatten(),
        _pdistB.loc[spgc_list, ref_list].values.flatten(),
    ),
)

plt.legend(markerscale=5)

In [ ]:
bins = np.linspace(0, 1, num=200)

_pdistA = strain_geno_pdist
_pdistB = strain_gene_uhgg_pdist


# Remove the diagonal from "minimum distance".
_pdistA = _pdistA + np.eye(len(_pdistA))
_pdistB = _pdistB + np.eye(len(_pdistB))

plt.scatter(
    _pdistA.loc[ref_list, ref_list].min(),
    _pdistB.loc[ref_list, ref_list].min(),
    s=10,
    alpha=0.5,
    label="ref-to-ref",
)
plt.scatter(
    _pdistA.loc[spgc_list, spgc_list].min(),
    _pdistB.loc[spgc_list, spgc_list].min(),
    s=10,
    alpha=0.5,
    label="spgc-to-spgc",
)
plt.scatter(
    _pdistA.loc[spgc_list, ref_list].min(),
    _pdistB.loc[spgc_list, ref_list].min(),
    s=10,
    alpha=0.5,
    label="ref-to-spgc",
)
plt.scatter(
    _pdistA.loc[ref_list, spgc_list].min(),
    _pdistB.loc[ref_list, spgc_list].min(),
    s=10,
    alpha=0.5,
    label="spgc-to-ref",
)
plt.xlabel("minimum_genotype_diss")
plt.ylabel("minimum_gene_diss")

plt.legend()

In [ ]:
_pdistA = strain_geno_pdist
_pdistB = strain_gene_uhgg_pdist

plt.scatter(
    squareform(_pdistA.loc[ref_list, ref_list]),
    squareform(_pdistB.loc[ref_list, ref_list]),
    s=1,
    alpha=0.5,
    color="k",
    label="ref-ref",
)

for spgc_strain_id in spgc_list:
    plt.scatter(
        _pdistA.loc[spgc_strain_id, ref_list],
        _pdistB.loc[spgc_strain_id, ref_list],
        s=1,
        alpha=0.5,
        color=spgc_palette[spgc_strain_id],
        # label='spgc-ref',
    )
    print(
        spgc_strain_id,
        sp.stats.pearsonr(
            _pdistA.loc[spgc_strain_id, ref_list], _pdistB.loc[spgc_strain_id, ref_list]
        ),
    )

plt.legend()

### Gene Filtering

#### Enriched/Depleted in SPGC Strains

In [ ]:
ref_gene_uhgg_prevalence = strain_gene_uhgg[ref_list].mean(1)  # NOTE: Set above
spgc_gene_uhgg_prevalence = strain_gene_uhgg[spgc_list].mean(1)

In [ ]:
x = ref_gene_uhgg_prevalence
y = spgc_gene_uhgg_prevalence

print(sp.stats.pearsonr(x, y))

fig, axs = plt.subplots(2, figsize=(5, 10))

bins0 = np.linspace(0.0, 1.0, num=50)
axs[0].hist2d(x, y, bins=bins0, norm=mpl.colors.PowerNorm(1 / 3, vmin=0, vmax=1e3))

bins1 = np.linspace(0.1, 0.9, num=40)
axs[1].hist2d(x, y, bins=bins1, norm=mpl.colors.PowerNorm(1 / 3))
axs[1].set_xlabel("reference prevalence")
axs[1].set_ylabel("inferred prevalence")
None

In [ ]:
bins = np.logspace(-4, 4)
plt.hist(
    (spgc_gene_uhgg_prevalence / ref_gene_uhgg_prevalence).replace(
        {0: 1e-3, np.inf: 1e3}
    ),
    bins=bins,
    alpha=0.5,
)
plt.hist(
    (ref_gene_uhgg_prevalence / spgc_gene_uhgg_prevalence).replace(
        {0: 1e-3, np.inf: 1e3}
    ),
    bins=bins,
    alpha=0.5,
)
plt.xscale("log")
plt.yscale("log")

In [ ]:
spgc_extremely_enriched = idxwhere(
    (spgc_gene_uhgg_prevalence / ref_gene_uhgg_prevalence) > 1000
)
spgc_extremely_depleted = idxwhere(
    (ref_gene_uhgg_prevalence / spgc_gene_uhgg_prevalence) > 1000
)
spgc_very_enriched = idxwhere(
    (spgc_gene_uhgg_prevalence / ref_gene_uhgg_prevalence) > 100
)
spgc_very_depleted = idxwhere(
    (ref_gene_uhgg_prevalence / spgc_gene_uhgg_prevalence) > 100
)
spgc_highly_enriched = idxwhere(
    (spgc_gene_uhgg_prevalence / ref_gene_uhgg_prevalence) > 10
)
spgc_highly_depleted = idxwhere(
    (ref_gene_uhgg_prevalence / spgc_gene_uhgg_prevalence) > 10
)
spgc_enriched = idxwhere((spgc_gene_uhgg_prevalence / ref_gene_uhgg_prevalence) > 5)
spgc_depleted = idxwhere((ref_gene_uhgg_prevalence / spgc_gene_uhgg_prevalence) > 5)
spgc_similar = idxwhere(
    ((spgc_gene_uhgg_prevalence / ref_gene_uhgg_prevalence) < 5)
    & ((ref_gene_uhgg_prevalence / spgc_gene_uhgg_prevalence) < 5)
)

#### Gene Length / Singletons

In [ ]:
for spgc_enrichment_class_list in [
    spgc_extremely_enriched,
    spgc_very_enriched,
    spgc_highly_enriched,
    spgc_enriched,
    spgc_similar,
    spgc_depleted,
    spgc_highly_depleted,
    spgc_very_depleted,
    spgc_extremely_depleted,
]:
    print(
        len(spgc_enrichment_class_list),
        uhgg_gene_length.loc[spgc_enrichment_class_list].mean(),
        uhgg_gene_length.loc[spgc_enrichment_class_list].std(),
    )

In [ ]:
short_genes = idxwhere(uhgg_gene_length < 300)
singleton_genes = idxwhere(strain_gene_uhgg.sum(1) <= 1)

#### Final Filter

In [ ]:
drop_list = spgc_highly_enriched + spgc_highly_depleted + short_genes + singleton_genes
print(
    len(spgc_highly_enriched),
    len(spgc_highly_depleted),
    len(short_genes),
    len(singleton_genes),
)

strain_uhgg_filt = strain_gene_uhgg.drop(index=drop_list, errors="ignore")
print(strain_gene_uhgg.shape[0], strain_uhgg_filt.shape[0])

In [ ]:
bins = np.linspace(0, strain_gene_uhgg[spgc_list + ref_list].sum().max(), num=50)

plt.hist(
    strain_gene_uhgg[ref_list].sum(),
    bins=bins,
    histtype="stepfilled",
    label="ref (uhgg)",
    density=True,
    color="tab:blue",
    alpha=0.5,
)
plt.hist(
    strain_gene_uhgg[spgc_list].sum(),
    bins=bins,
    histtype="stepfilled",
    label="spgc (uhgg)",
    density=True,
    color="tab:orange",
    alpha=0.5,
)

plt.hist(
    strain_uhgg_filt[ref_list].sum(),
    bins=bins,
    histtype="step",
    label="ref (filtered uhgg)",
    density=True,
    linestyle="-",
    color="tab:blue",
    lw=2,
)
plt.hist(
    strain_uhgg_filt[spgc_list].sum(),
    bins=bins,
    histtype="step",
    label="spgc (filtered uhgg)",
    density=True,
    linestyle="-",
    color="tab:orange",
    lw=2,
)

# plt.hist(strain_gene_eggnog[ref_list].sum(), bins=bins, histtype='stepfilled', label='ref (eggnog)', density=True, color='tab:blue', alpha=0.3)
# plt.hist(strain_gene_eggnog[spgc_list].sum(), bins=bins, histtype='stepfilled', label='spgc (eggnog)', density=True, color='tab:orange', alpha=0.3)

# plt.hist(strain_gene_filt_eggnog[ref_list].sum(), bins=bins, histtype='step', label='ref (filtered eggnog)', density=True, linestyle='--', color='tab:blue', lw=1.5)
# plt.hist(strain_gene_filt_eggnog[spgc_list].sum(), bins=bins, histtype='step', label='spgc (filtered eggnog)', density=True, linestyle='--', color='tab:orange', lw=1.5)

plt.legend()

None

In [ ]:
uhgg_filt_cdmat = sp.spatial.distance.pdist(strain_uhgg_filt, metric="cosine")
uhgg_filt_pdist = pd.DataFrame(
    sp.spatial.distance.squareform(uhgg_filt_cdmat),
    index=strain_uhgg_filt.index,
    columns=strain_uhgg_filt.index,
)

In [ ]:
uhgg_filt_linkage = sp.cluster.hierarchy.linkage(uhgg_filt_cdmat, method="average")
uhgg_filt_linkage.shape

In [ ]:
strain_uhgg_filt_cdmat = sp.spatial.distance.pdist(
    strain_uhgg_filt.T,
    metric="cosine",
    w=ref_gene_uhgg_entropy.loc[strain_uhgg_filt.index],
)
strain_uhgg_filt_pdist = pd.DataFrame(
    sp.spatial.distance.squareform(strain_uhgg_filt_cdmat),
    index=strain_uhgg_filt.columns,
    columns=strain_uhgg_filt.columns,
)
strain_uhgg_filt_linkage = sp.cluster.hierarchy.linkage(
    strain_uhgg_filt_cdmat, method="average", optimal_ordering=True
)

In [ ]:
strain_uhgg_filt_unweighted_cdmat = sp.spatial.distance.pdist(
    strain_uhgg_filt.T, metric="cosine"
)
strain_uhgg_filt_unweighted_pdist = pd.DataFrame(
    sp.spatial.distance.squareform(strain_uhgg_filt_unweighted_cdmat),
    index=strain_uhgg_filt.columns,
    columns=strain_uhgg_filt.columns,
)
strain_uhgg_filt_unweighted_linkage = sp.cluster.hierarchy.linkage(
    strain_uhgg_filt_unweighted_cdmat, method="average", optimal_ordering=True
)

In [ ]:
x = strain_uhgg_filt
_col_linkage = strain_geno_linkage
_row_linkage = uhgg_filt_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)

sns.clustermap(x, row_cluster=False, col_linkage=_col_linkage, col_colors=_col_colors)

In [ ]:
x = strain_uhgg_filt
_col_linkage = strain_uhgg_filt_linkage
_row_linkage = uhgg_filt_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)

sns.clustermap(x, row_cluster=False, col_linkage=_col_linkage, col_colors=_col_colors)

In [ ]:
x = strain_uhgg_filt
_col_linkage = strain_uhgg_filt_unweighted_linkage
_row_linkage = uhgg_filt_linkage
# Order x by leaf order.
# See <https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.ClusterNode.pre_order.html#scipy.cluster.hierarchy.ClusterNode.pre_order>
x = x.iloc[sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)]

_col_colors = genome_type.map(genome_type_palette)

sns.clustermap(x, row_cluster=False, col_linkage=_col_linkage, col_colors=_col_colors)

### Gene Content Divergence from Refs

Are SPGC Genomes Believably Distinct?

Test if SPGC Strains are surprisingly distinct from refs in gene content relative to their distinction in genotype

#### Min-genotype / Min-gene content

In [ ]:
_pdistA = strain_geno_pdist
_pdistB = strain_uhgg_filt_pdist

plt.scatter(
    squareform(_pdistA.loc[ref_list, ref_list]),
    squareform(_pdistB.loc[ref_list, ref_list]),
    s=1,
    alpha=0.5,
    label="ref-ref",
)
print(
    "ref-ref",
    sp.stats.pearsonr(
        squareform(_pdistA.loc[ref_list, ref_list]),
        squareform(_pdistB.loc[ref_list, ref_list]),
    ),
)

plt.scatter(
    squareform(_pdistA.loc[spgc_list, spgc_list]),
    squareform(_pdistB.loc[spgc_list, spgc_list]),
    s=1,
    alpha=0.5,
    label="spgc-spgc",
)
print(
    "spgc-spgc",
    sp.stats.pearsonr(
        squareform(_pdistA.loc[spgc_list, spgc_list]),
        squareform(_pdistB.loc[spgc_list, spgc_list]),
    ),
)

plt.scatter(
    _pdistA.loc[spgc_list, ref_list].values.flatten(),
    _pdistB.loc[spgc_list, ref_list].values.flatten(),
    s=1,
    alpha=0.5,
    label="spgc-ref",
)
print(
    "spgc-ref",
    sp.stats.pearsonr(
        _pdistA.loc[spgc_list, ref_list].values.flatten(),
        _pdistB.loc[spgc_list, ref_list].values.flatten(),
    ),
)

plt.legend(markerscale=5)

In [ ]:
_pdistA = strain_geno_pdist
_pdistB = strain_uhgg_filt_pdist

plt.scatter(
    squareform(_pdistA.loc[ref_list, ref_list]),
    squareform(_pdistB.loc[ref_list, ref_list]),
    s=1,
    alpha=0.5,
    color='k',
    label='ref-ref',
)

for spgc_strain_id in spgc_list:
    plt.scatter(
        _pdistA.loc[spgc_strain_id, ref_list],
        _pdistB.loc[spgc_strain_id, ref_list],
        s=1,
        alpha=0.5,
        color=spgc_palette[spgc_strain_id],
        # label='spgc-ref',
    )
    print(spgc_strain_id, sp.stats.pearsonr(_pdistA.loc[spgc_strain_id, ref_list], _pdistB.loc[spgc_strain_id, ref_list]))

plt.legend()

In [ ]:
_pdistA = strain_geno_pdist
_pdistB = strain_gene_uhgg_pdist

plt.scatter(
    squareform(_pdistA.loc[ref_list, ref_list]),
    squareform(_pdistB.loc[ref_list, ref_list]),
    s=1,
    alpha=0.5,
    color='k',
    label='ref-ref',
)

for spgc_strain_id in spgc_list:
    plt.scatter(
        _pdistA.loc[spgc_strain_id, ref_list],
        _pdistB.loc[spgc_strain_id, ref_list],
        s=1,
        alpha=0.5,
        color=spgc_palette[spgc_strain_id],
        # label='spgc-ref',
    )
    print(spgc_strain_id, sp.stats.pearsonr(_pdistA.loc[spgc_strain_id, ref_list], _pdistB.loc[spgc_strain_id, ref_list]))

plt.legend()

#### Min-genotype / Matched-gene content

In [ ]:
_pdistA = strain_geno_pdist
_pdistB = strain_uhgg_filt_pdist

geno_pdist_adjust = 1 / sfacts_fit.sizes["position"]
# gene_pdist_adjust = (1 / strain_gene_filt_eggnog.shape[0])


# Remove the diagonal from "minimum distance".
_pdistA = _pdistA + np.eye(len(_pdistA))
_pdistB = _pdistB + np.eye(len(_pdistB))

isolate_to_ref_min = _pdistA.loc[ref_list, isolate_list].idxmin()
isolate_to_ref_min_pdistA = _pdistA.stack()[
    isolate_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)
isolate_to_ref_min_pdistB = _pdistB.stack()[
    isolate_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)

mag_to_ref_min = _pdistA.loc[ref_list, mag_list].idxmin()
mag_to_ref_min_pdistA = _pdistA.stack()[
    mag_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)
mag_to_ref_min_pdistB = _pdistB.stack()[
    mag_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)

spgc_to_ref_min = _pdistA.loc[ref_list, spgc_list].idxmin()
spgc_to_ref_min_pdistA = _pdistA.stack()[
    spgc_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)
spgc_to_ref_min_pdistB = _pdistB.stack()[
    spgc_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)

d0 = pd.DataFrame(
    dict(
        min_geno_ref=pd.concat(
            [
                isolate_to_ref_min,
                mag_to_ref_min,
                spgc_to_ref_min,
            ]
        ),
        geno_dist=(
            pd.concat(
                [
                    isolate_to_ref_min_pdistA,
                    mag_to_ref_min_pdistA,
                    spgc_to_ref_min_pdistA,
                ]
            )
            + geno_pdist_adjust
        ),
        gene_dist=pd.concat(
            [isolate_to_ref_min_pdistB, mag_to_ref_min_pdistB, spgc_to_ref_min_pdistB]
        ),
        genome_type=["Isolate"] * len(isolate_to_ref_min_pdistA)
        + ["MAG"] * len(mag_to_ref_min_pdistA)
        + ["SPGC"] * len(spgc_to_ref_min_pdistA),
    )
)  # .assign(
#     geno_dist_pc=lambda x: x.geno_dist + geno_pdist_adjust,
#     gene_dist_pc=lambda x: x.gene_dist + gene_pdist_adjust,
# )
fit = smf.ols(f"gene_dist ~ np.log2(geno_dist) * genome_type", data=d0).fit()
genome_distance_comparison = d0.assign(
    gene_dist_predict=lambda x: fit.predict(), gene_dist_resid_pearson=fit.resid_pearson
).sort_values("geno_dist")

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
for _genome_type, d2 in genome_distance_comparison.groupby("genome_type"):
    ax.scatter("geno_dist", "gene_dist", data=d2, label=_genome_type)
    ax.plot("geno_dist", "gene_dist_predict", data=d2, label="__nolegend__")

ax = axs[1]
for _genome_type, d2 in genome_distance_comparison.groupby("genome_type"):
    ax.scatter("geno_dist", "gene_dist", data=d2, label=_genome_type)
    ax.plot("geno_dist", "gene_dist_predict", data=d2, label="__nolegend__")
ax.legend()
ax.set_xscale("symlog", linthresh=1e-4)
fit.summary()

In [ ]:
genome_type_to_label = {"isolate": "Isolate", "mag": "MAG", "spgc": "SPGC"}

fig, ax = plt.subplots(figsize=(5, 5))
for _genome_type, d2 in genome_distance_comparison.groupby("genome_type"):
    ax.scatter(
        "geno_dist",
        "gene_dist",
        data=d2,
        color=genome_type_palette[_genome_type],
        label=_genome_type,
        s=50,
        alpha=0.7,
        lw=0,
    )
    ax.plot(
        "geno_dist",
        "gene_dist_predict",
        data=d2,
        color=genome_type_palette[_genome_type],
        label="__nolegend__",
    )
# ax.legend(markerscale=3)
ax.set_xscale("symlog", linthresh=1e-4)
ax.set_xlabel("SNP Profile Dissimilarity")
ax.set_ylabel("Gene Content Dissimilarity")
# ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4])

In [ ]:
_pdistB = strain_uhgg_filt_unweighted_pdist
_pdistB = _pdistB + np.eye(len(_pdistB))

isolate_to_ref_min_pdistB_raw = _pdistB.stack()[
    isolate_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)
mag_to_ref_min_pdistB_raw = _pdistB.stack()[
    mag_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)
spgc_to_ref_min_pdistB_raw = _pdistB.stack()[
    spgc_to_ref_min.reset_index().set_index(["index", 0]).index
].reset_index(level=1, drop=True)

d0 = pd.DataFrame(
    dict(
        min_geno_ref=pd.concat(
            [
                isolate_to_ref_min,
                mag_to_ref_min,
                spgc_to_ref_min,
            ]
        ),
        geno_dist=(
            pd.concat(
                [
                    isolate_to_ref_min_pdistA,
                    mag_to_ref_min_pdistA,
                    spgc_to_ref_min_pdistA,
                ]
            )
            + geno_pdist_adjust
        ),
        gene_dist=pd.concat(
            [
                isolate_to_ref_min_pdistB_raw,
                mag_to_ref_min_pdistB_raw,
                spgc_to_ref_min_pdistB_raw,
            ]
        ),
        genome_type=["Isolate"] * len(isolate_to_ref_min_pdistA)
        + ["MAG"] * len(mag_to_ref_min_pdistA)
        + ["SPGC"] * len(spgc_to_ref_min_pdistA),
    )
)
fit = smf.ols(f"gene_dist ~ np.log2(geno_dist) * genome_type", data=d0).fit()
genome_distance_comparison_raw = d0.assign(
    gene_dist_predict=lambda x: fit.predict(), gene_dist_resid_pearson=fit.resid_pearson
).sort_values("geno_dist")

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
for _genome_type, d2 in genome_distance_comparison_raw.groupby("genome_type"):
    ax.scatter("geno_dist", "gene_dist", data=d2, label=_genome_type)
    ax.plot("geno_dist", "gene_dist_predict", data=d2, label="__nolegend__")

ax = axs[1]
for _genome_type, d2 in genome_distance_comparison_raw.groupby("genome_type"):
    ax.scatter("geno_dist", "gene_dist", data=d2, label=_genome_type)
    ax.plot("geno_dist", "gene_dist_predict", data=d2, label="__nolegend__")
ax.legend()
ax.set_xscale("symlog", linthresh=1e-4)
fit.summary()

In [ ]:
genome_type_to_label = {"isolate": "Isolate", "mag": "MAG", "spgc": "SPGC"}

fig, ax = plt.subplots(figsize=(5, 5))
for _genome_type, d2 in genome_distance_comparison_raw.groupby("genome_type"):
    ax.scatter(
        "geno_dist",
        "gene_dist",
        data=d2,
        color=genome_type_palette[_genome_type],
        label=_genome_type,
        s=50,
        alpha=0.7,
        lw=0,
    )
    ax.plot(
        "geno_dist",
        "gene_dist_predict",
        data=d2,
        color=genome_type_palette[_genome_type],
        label="__nolegend__",
    )
# ax.legend(markerscale=3)
ax.set_xscale("symlog", linthresh=1e-4)
ax.set_xlabel("SNP Profile Dissimilarity")
ax.set_ylabel("Gene Content Dissimilarity")
# ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4])

## Strain Diversity Analysis

In [ ]:
x = strain_geno_pdist.replace({0: np.nan})

_colors = pd.DataFrame(
    dict(
        g=genome_type.map(genome_type_palette),
    )
)

g = sns.clustermap(
    x,
    row_colors=_colors,
    col_colors=_colors,
    row_linkage=strain_uhgg_filt_linkage,
    col_linkage=strain_geno_linkage,
    xticklabels=0,
    yticklabels=0,
    dendrogram_ratio=0.1,
    vmin=0,
    vmax=1,
    tree_kws=dict(lw=1.0),
    # figsize=(40, 40),
)
g.ax_heatmap.set_facecolor("aqua")
g.cax.set_visible(False)

In [ ]:
x = strain_uhgg_filt_pdist.replace({0: np.nan})

_colors = pd.DataFrame(
    dict(
        g=genome_type.map(genome_type_palette),
    )
)

g = sns.clustermap(
    x,
    row_colors=_colors,
    col_colors=_colors,
    row_linkage=strain_uhgg_filt_linkage,
    col_linkage=strain_geno_linkage,
    xticklabels=0,
    yticklabels=0,
    dendrogram_ratio=0.1,
    vmin=0,
    vmax=1,
    tree_kws=dict(lw=1.0),
    # figsize=(40, 40),
)
g.ax_heatmap.set_facecolor("aqua")
g.cax.set_visible(False)

### Prevalence Comparisons

In [ ]:
ref_uhgg_prevalence = strain_gene_uhgg[ref_list].mean(1)
spgc_uhgg_prevalence = strain_gene_uhgg[spgc_list].mean(1)

In [ ]:
x = ref_uhgg_prevalence
y = spgc_uhgg_prevalence

print(sp.stats.pearsonr(x, y))

fig, axs = plt.subplots(2, figsize=(5, 10))

bins0 = np.linspace(0.0, 1.0, num=50)
axs[0].hist2d(x, y, bins=bins0, norm=mpl.colors.PowerNorm(1 / 3, vmin=0, vmax=1e3))

bins1 = np.linspace(0.1, 0.9, num=40)
axs[1].hist2d(x, y, bins=bins1, norm=mpl.colors.PowerNorm(1 / 3))
axs[1].set_xlabel("reference prevalence")
axs[1].set_ylabel("inferred prevalence")
None

### Core / Shell / Cloud Pangenome

In [ ]:
bins = np.linspace(0, 1, num=51)


def _assign_prevalence_class(p):
    if p > 0.95:
        return "core"
    elif p > 0.1:
        return "shell"
    elif p < 0.1:
        return "cloud"


prevalence_class_order = ["core", "shell", "cloud"]
prevalence_class_palette = {
    "core": "tab:blue",
    "shell": "tab:orange",
    "cloud": "tab:green",
}

spgc_uhgg_class = spgc_uhgg_prevalence.map(_assign_prevalence_class)

for prevalence_class in prevalence_class_order:
    plt.hist(
        spgc_uhgg_prevalence[spgc_uhgg_class == prevalence_class],
        bins=bins,
        label=prevalence_class,
        color=prevalence_class_palette[prevalence_class],
    )
plt.legend()
plt.yscale("log")
# core_genes = idxwhere(strain_uhgg_filt_prevalence > 0.9)
# shell_genes = idxwhere((strain_uhgg_filt_prevalence < 0.9) & (strain_uhgg_filt_prevalence > 0.1))
# cloud_genes = idxwhere(strain_uhgg_filt_prevalence < 0.1)

In [ ]:
import matplotlib.gridspec as gridspec

d = strain_gene_uhgg.groupby(spgc_uhgg_class).sum().T[prevalence_class_order]

fig = plt.figure(figsize=(10, 5), tight_layout=True)

gs = gridspec.GridSpec(
    1, 2, width_ratios=[len(ref_list) / len(spgc_list), 1], figure=fig
)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1], sharey=ax0)

# , axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
for ax, strain_list, title in zip(
    [ax0, ax1], [ref_list, spgc_list], ["references", "spgc"]
):
    # TODO: Fix the colors to always match the previous palette.
    d.loc[strain_list].sort_values("shell").plot.bar(stacked=True, ax=ax, width=1)
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_aspect(0.08)
    ax.legend_.set_visible(False)
ax0.legend_.set_visible(True)

In [ ]:
gene_annotations.groupby(spgc_uhgg_class).centroid_99_length.quantile(
    [0.1, 0.25, 0.5, 0.75, 0.9]
).unstack()

In [ ]:
cog_category_order = gene_x_cog_category_matrix.columns
cog_category_palette = lib.plot.construct_ordered_palette(
    cog_category_order, cm="rainbow", extend=dict(no_category="grey")
)
for cog_category in cog_category_order:
    plt.scatter([], [], c=cog_category_palette[cog_category], label=cog_category)
plt.legend(ncols=4)

lib.plot.hide_axes_and_spines()

In [ ]:
cog_category_description = pd.read_table(
    "ref/cog-20.categories.tsv",
    names=["cog_category", "description"],
    index_col="cog_category",
).assign(description=lambda x: x.index + ": " + x.description)
cog_category_description.loc["no_category", "description"] = "-: No Annotation"
cog_category_description

In [ ]:
x = spgc_uhgg_class
y = gene_x_cog_category_matrix.reindex(
    x.index, fill_value=False
)  # .assign(no_category=lambda x: x.sum(1) == 0)
gene_list = spgc_uhgg_class.index

cog_category_gene_class_enrichment_test = []

for _prevalence_class, _cog_category in product(
    prevalence_class_order, list(cog_category_order)
):
    contingency_table = (
        pd.DataFrame(
            dict(
                is_prev_class=(x == _prevalence_class),
                is_cog_category=y[_cog_category],
            )
        )
        .value_counts()
        .unstack()
        .reindex(index=[False, True], columns=[False, True])
        .fillna(0)
    )
    _test = sp.stats.fisher_exact(contingency_table)
    cog_category_gene_class_enrichment_test.append(
        (
            _prevalence_class,
            _cog_category,
            _test[0],
            _test[1],
            contingency_table.loc[True, True],
        )
    )

cog_category_gene_class_enrichment_test = (
    pd.DataFrame(
        cog_category_gene_class_enrichment_test,
        columns=[
            "prevalence_class",
            "cog_category",
            "statistic",
            "pvalue",
            "gene_count",
        ],
    )
    .set_index(["prevalence_class", "cog_category"])
    .assign(
        negative_log10_pvalue=lambda x: -np.log10(x.pvalue),
        log2_odds_ratio=lambda x: np.log2(x.statistic),
    )
)

In [ ]:
def _assign_significance_marker(pvalue):
    if pvalue < 1e-5:
        return "***"
    elif pvalue < 1e-3:
        return "**"
    elif pvalue < 0.05:
        return "*"
    else:
        return ""

In [ ]:
x = (
    cog_category_gene_class_enrichment_test.log2_odds_ratio.unstack("prevalence_class")
    .replace({np.inf: np.nan, -np.inf: np.nan})
    .join(cog_category_description)
    .set_index("description")[prevalence_class_order]
    .fillna(0)
)
# annot = (cog_category_gene_class_enrichment_test.pvalue.map(_assign_significance_marker) + '|' + cog_category_gene_class_enrichment_test.gene_count.astype(int).astype(str)).unstack('prevalence_class')[prevalence_class_order]
annot = (
    cog_category_gene_class_enrichment_test.pvalue.map(_assign_significance_marker)
    .unstack("prevalence_class")
    .join(cog_category_description)
    .set_index("description")[prevalence_class_order]
)
# annot = cog_category_gene_class_enrichment_test.gene_count.unstack('prevalence_class')[prevalence_class_order].astype(int)

_row_order = x["core"].sort_values(ascending=False).index
# x, annot = lib.pandas_util.align_indexes(x, annot)

fig, ax = plt.subplots(figsize=(5, 12))
ax = sns.heatmap(
    x.reindex(_row_order),
    annot=annot.reindex(_row_order),
    fmt="",
    cmap="coolwarm",
    center=0,
    vmin=-5,
    vmax=5,
    cbar_kws=dict(
        use_gridspec=True, location="left", label="log2(odds ratio)", extend="both"
    ),
    ax=ax,
    yticklabels=1,
    xticklabels=1,
    annot_kws=dict(va="center"),
    # norm=mpl.colors.SymLogNorm(linthresh=1e1),
    # center=0,
)

ax.yaxis.set_ticks_position("right")
ax.set_ylabel("")
lib.plot.rotate_yticklabels(rotation=-0, va="center")

In [ ]:
fig, ax = plt.subplots()

for prevalence_class in prevalence_class_order:
    d = cog_category_gene_class_enrichment_test.xs(
        prevalence_class, level="prevalence_class"
    )
    ax.scatter(
        "log2_odds_ratio",
        "negative_log10_pvalue",
        data=d,
        c=prevalence_class_palette[prevalence_class],
        label=prevalence_class,
        s=5,
    )

# Annotate top-10
for idx in (
    cog_category_gene_class_enrichment_test.sort_values(
        "negative_log10_pvalue", ascending=False
    )
    .head(10)
    .index
):
    ax.annotate(
        idx[1],
        xy=cog_category_gene_class_enrichment_test.loc[idx][
            ["log2_odds_ratio", "negative_log10_pvalue"]
        ],
        xytext=(2, 1),
        textcoords="offset pixels",
        color=prevalence_class_palette[idx[0]],
        # ha='center',
        # va='center',
        fontweight="bold",
    )
ax.axvline(0, linestyle="--", lw=1, color="k")
ax.axhline(
    -np.log10(0.05 / cog_category_gene_class_enrichment_test.shape[0]),
    linestyle="--",
    lw=1,
    color="k",
)
ax.legend()
# cog_category_gene_class_enrichment_test.sort_values('negative_log10_pvalue', ascending=False).head(20)

### Strength of Phylogenetic Signal in Shell Genes

In [ ]:
shell_gene_list = idxwhere(spgc_uhgg_class == "shell")

x = strain_gene_uhgg.loc[shell_gene_list]

strain_shell_gene_cdist = sp.spatial.distance.pdist(
    x.T, metric="cosine", w=ref_gene_uhgg_entropy.loc[x.index]
)
strain_shell_gene_pdist = pd.DataFrame(
    squareform(strain_shell_gene_cdist), index=x.columns, columns=x.columns
)
strain_shell_gene_linkage = sp.cluster.hierarchy.linkage(
    strain_shell_gene_cdist, method="average", optimal_ordering=True
)

# _col_linkage = strain_shell_gene_linkage
_col_colors = genome_type.map(genome_type_palette)
_col_linkage = strain_shell_gene_linkage

sns.clustermap(x, col_linkage=_col_linkage, col_colors=_col_colors)

In [ ]:
from scipy.spatial.distance import pdist, squareform

# FIXME: x is ambiguous here.
x = strain_shell_gene_pdist.replace({0: np.nan})
_col_linkage = strain_geno_linkage
_row_linkage = strain_shell_gene_linkage
_colors = genome_type.map(genome_type_palette)

g = sns.clustermap(
    x,
    col_linkage=_col_linkage,
    row_linkage=_row_linkage,
    col_colors=_colors,
    row_colors=_colors,
)
g.ax_heatmap.set_facecolor("aqua")

In [ ]:
phylogenetic_signal_spgc = {}
phylogenetic_signal_ref = {}
for gene_id in tqdm(strain_gene_uhgg.index):
    
    gene_pdist_spgc = pdist(strain_gene_uhgg.loc[[gene_id], spgc_list].T)
    if gene_pdist_spgc.mean() != 0:
        geno_pdist_spgc = squareform(strain_geno_pdist.loc[spgc_list, spgc_list])
        phylogenetic_signal_spgc[gene_id] = sp.stats.pearsonr(geno_pdist_spgc, gene_pdist_spgc)
    else:
        phylogenetic_signal_spgc[gene_id] = np.nan

    gene_pdist_ref = pdist(strain_gene_uhgg.loc[[gene_id], ref_list].T)
    if gene_pdist_ref.mean() != 0:
        geno_pdist_ref = squareform(strain_geno_pdist.loc[ref_list, ref_list])
        phylogenetic_signal_ref[gene_id] = sp.stats.pearsonr(geno_pdist_ref, gene_pdist_ref)
    else:
        phylogenetic_signal_ref[gene_id] = np.nan

phylogenetic_signal_spgc = pd.DataFrame(
    phylogenetic_signal_spgc, index=["r_statistic", "pvalue"]
).T
phylogenetic_signal_ref = pd.DataFrame(
    phylogenetic_signal_ref, index=["r_statistic", "pvalue"]
).T

In [ ]:
phylogenetic_signal_spgc.sort_values("r_statistic")
plt.plot(phylogenetic_signal_spgc.r_statistic.sort_values().values)

In [ ]:
phylogenetic_signal_ref

In [ ]:
d = pd.DataFrame(dict(x=phylogenetic_signal_ref.r_statistic, y=phylogenetic_signal_spgc.r_statistic)).loc[shell_gene_list].dropna()

print(sp.stats.pearsonr(d.x, d.y))

sns.jointplot(
    x='x', y='y',
    data=d,
    kind='reg',
)

In [ ]:
x = strain_gene_uhgg.loc[shell_gene_list]  # , spgc_list]
_col_colors = genome_type.map(genome_type_palette)
_row_colors = pd.DataFrame(
    dict(
        phylo=phylogenetic_signal_spgc.r_statistic.map(
            lambda x: mpl.cm.coolwarm((x + 1) / 2)
        ),
        entrp=ref_gene_uhgg_entropy.map(mpl.cm.viridis),
    )
)

sns.clustermap(
    x, row_colors=_row_colors, col_linkage=strain_geno_linkage, col_colors=_col_colors
)

In [ ]:
is_phylogenetic = pd.concat(
    [
        pd.Series(
            True,
            index=phylogenetic_signal_spgc[
                lambda x: (x.r_statistic > 0.1) & (x.pvalue < 0.001)
            ].index,
        ),
        pd.Series(
            False,
            index=phylogenetic_signal_spgc[
                lambda x: (x.r_statistic < 0.1) & (x.pvalue > 0.05)
            ].index,
        ),
    ]
)

cog_category_phylogenetic_enrichment_test = []

for _cog_category in tqdm(list(cog_category_order)):
    contingency_table = (
        pd.DataFrame(
            dict(
                is_phylogenetic=is_phylogenetic,
                is_cog_category=gene_x_cog_category_matrix[_cog_category],
            )
        )
        .dropna()
        .value_counts()
        .unstack()
        .reindex(index=[False, True], columns=[False, True])
        .fillna(0)
    )
    _test = sp.stats.fisher_exact(contingency_table)
    cog_category_phylogenetic_enrichment_test.append(
        (_cog_category, *_test, *contingency_table.stack())
    )

cog_category_phylogenetic_enrichment_test = (
    pd.DataFrame(
        cog_category_phylogenetic_enrichment_test,
        columns=["cog_category", "statistic", "pvalue", "np_nc", "np_c", "p-nc", "p-c"],
    )
    .set_index(["cog_category"])
    .assign(
        negative_log10_pvalue=lambda x: -np.log10(x.pvalue),
        log2_odds_ratio=lambda x: np.log2(x.statistic),
    )
)

In [ ]:
cog_category_phylogenetic_enrichment_test.sort_values(
    "log2_odds_ratio", ascending=False
).join(cog_category_description)

In [ ]:
cog_category_phylogenetic_signal_test = {}

for _cog_category in tqdm(list(cog_category_order)):
    d = pd.DataFrame(
        dict(
            phylogenetic_signal=phylogenetic_signal_spgc.r_statistic,
            is_cog_category=gene_x_cog_category_matrix.reindex(
                phylogenetic_signal_spgc.index
            ).fillna(False)[_cog_category],
        )
    ).loc[shell_gene_list]
    x = d[d.is_cog_category].phylogenetic_signal
    y = d[~d.is_cog_category].phylogenetic_signal
    if (len(x) > 0) and (len(y) > 0):
        _test = sp.stats.mannwhitneyu(x, y)
    else:
        _test = (np.nan, np.nan)
    cog_category_phylogenetic_signal_test[_cog_category] = (
        len(x),
        x.median(),
        y.median(),
        x.mean(),
        y.mean(),
        *_test,
    )

cog_category_phylogenetic_signal_test = pd.DataFrame(
    cog_category_phylogenetic_signal_test,
    index=[
        "num_genes_in_category",
        "cog_median_r",
        "not_cog_median_r",
        "cog_mean_r",
        "not_cog_mean_r",
        "mwu_statistic",
        "pvalue",
    ],
).T.assign(
    negative_log10_pvalue=lambda x: -np.log10(x.pvalue),
    median_diff=lambda x: x.cog_median_r - x.not_cog_median_r,
)

In [ ]:
d = cog_category_phylogenetic_signal_test.join(cog_category_description).dropna(
    subset=["median_diff"]
)

fig, ax = plt.subplots()
plt.scatter(
    "median_diff",
    "pvalue",
    data=d,
)
# lib.plot.rotate_xticklabels()
ax.axvline(0, linestyle="--", lw=1, color="k")
# ax.axhline(-np.log10(0.05 / d.shape[0]), linestyle='--', lw=1, color='k')


for cog_category in idxwhere((np.abs(d.median_diff) > 0) & (d.pvalue < 1e-4)):
    plt.annotate(
        cog_category,
        xy=d[["median_diff", "pvalue"]].loc[cog_category],
        xytext=(4, 2),
        textcoords="offset points",
        # color=prevalence_class_palette[prevalence_class],
        # ha='center',
        # va='center',
        # fontweight='bold',
    )

plt.yscale("log")
plt.xlim(-0.2, 0.2)
plt.ylim(1e-15, 1)
plt.gca().invert_yaxis()
plt.xlabel("Difference in Median $r$\n(in COG category vs. not)")
plt.ylabel("P-value")

d.sort_values("median_diff", ascending=False)

In [ ]:
x = (
    cog_category_phylogenetic_signal_test.join(cog_category_description)[
        ["cog_median_r"]
    ]
    .dropna()
    .join(cog_category_description)
    .set_index("description")
)

# Order rows based on the COG-category enrichment analysis
_row_order = (
    cog_category_gene_class_enrichment_test.log2_odds_ratio.unstack("prevalence_class")
    .replace({np.inf: np.nan, -np.inf: np.nan})
    .join(cog_category_description)
    .set_index("description")[prevalence_class_order]
    .fillna(0)["core"]
    .sort_values(ascending=False)
    .index
)
annot = (
    cog_category_phylogenetic_signal_test.pvalue.map(_assign_significance_marker)
    .to_frame()
    .join(cog_category_description)
    .set_index("description")
)


fig, ax = plt.subplots(figsize=(3.5, 12))
ax = sns.heatmap(
    x.reindex(_row_order),
    annot=annot.reindex(_row_order),
    fmt="",
    cmap="PuOr",
    center=phylogenetic_signal_spgc.loc[shell_gene_list].r_statistic.mean(),
    cbar_kws=dict(
        use_gridspec=True,
        location="left",
        label="Medians $r$",
        extend="both",
        fraction=0.5,
    ),
    # ax=ax,
    # yticklabels=1,
    # xticklabels=1,
    # annot_kws=dict(va='center'),
    # norm=mpl.colors.SymLogNorm(linthresh=1e1),
)

ax.yaxis.set_ticks_position("right")
ax.set_ylabel("")
lib.plot.rotate_yticklabels(rotation=-0, va="center")

In [ ]:
d = phylogenetic_signal_spgc.loc[shell_gene_list].r_statistic.dropna().to_frame().join(gene_x_cog_category).fillna("-")

fig, ax = plt.subplots(figsize=(20, 5))
sns.violinplot(
    data=d,
    x="cog_category",
    y="r_statistic",
    ax=ax,
    order=cog_category_phylogenetic_signal_test.cog_median_r.sort_values().index,
)

## Gene Co-occurence

### Found using SPGC only

In [ ]:
gene_prevalence = strain_uhgg_filt[spgc_list].mean(1)
plt.hist(gene_prevalence)
plt.yscale("log")
variable_genes = idxwhere((gene_prevalence > 0.05) & (gene_prevalence < 0.95))
len(variable_genes)

In [ ]:
x = strain_gene_uhgg
y = x[spgc_list]

drop_ubiq_genes_list = idxwhere(y.mean(1) == 1)
drop_nohit_genes_list = idxwhere(y.mean(1) == 0)
z = y.drop(drop_ubiq_genes_list + drop_nohit_genes_list)

_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)

clust1 = pd.Series(
    AgglomerativeClustering(
        n_clusters=None,
        metric="correlation",
        linkage="average",
        distance_threshold=0.035,
    ).fit_predict(z),
    index=z.index,
)
# Add back ubiquitous and nohit genes as clusters.
clust1 = pd.concat(
    [
        clust1,
        pd.Series(-1, index=drop_ubiq_genes_list),
        pd.Series(-2, index=drop_nohit_genes_list),
    ]
)
clust1_palette = lib.plot.construct_ordered_palette(clust1.unique())

sns.clustermap(
    x.loc[variable_genes],
    row_colors=x.loc[variable_genes].index.to_series().map(clust1).map(clust1_palette),
    col_linkage=strain_geno_linkage,
    col_colors=_col_colors,
    metric="cosine",
)

In [ ]:
clust1_sizes = clust1.value_counts()

clust1_sizes_meta = (
    clust1.value_counts().to_frame("tally")
    # .groupby('clust')
    # .count()
    # .sort_values(ascending=False)
    .assign(
        top5_cc=gene_x_cog_category_matrix.groupby(clust1)
        .sum()
        .rename(int)
        .astype(int)
        .apply(lambda x: x.sort_values(ascending=False).head(5).index.to_list(), axis=1)
    )
)

clust1_sizes_meta.head(10)

In [ ]:
_clust = clust1_sizes_meta.index[9]

_gene_list = idxwhere(clust1 == _clust)
x = strain_gene_uhgg.loc[_gene_list]
y = strain_gene_uhgg_depth.loc[_gene_list]
_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)
_row_colors = (
    gene_annotations.reindex(_gene_list)
    .centroid_99_length.map(np.log10)
    .map(lambda x: x / 4)
    .map(mpl.cm.viridis)
)
_col_linkage = strain_geno_linkage
_row_linkage = sp.cluster.hierarchy.linkage(
    x, method="average", metric="cosine", optimal_ordering=True
)  # TODO

sns.clustermap(
    x,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    xticklabels=0,
    yticklabels=0,
)
sns.clustermap(
    y,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    norm=mpl.colors.PowerNorm(1 / 2, vmin=0, vmax=2),
    xticklabels=0,
    yticklabels=0,
)

print(
    gene_x_cog_category_matrix.reindex(_gene_list)
    .sum()
    .sort_values(ascending=False)[lambda x: x > 0]
    .to_frame("tally")
    .join(cog_category_description)
)
gene_annotations.reindex(_gene_list)

### Found using SPGC and Ref

In [ ]:
x = strain_gene_uhgg
y = x[spgc_list + ref_list]

drop_ubiq_genes_list = idxwhere(y.mean(1) == 1)
drop_nohit_genes_list = idxwhere(y.mean(1) == 0)
z = y.drop(drop_ubiq_genes_list + drop_nohit_genes_list)

_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)

clust2 = pd.Series(
    AgglomerativeClustering(
        n_clusters=None,
        metric="correlation",
        linkage="average",
        distance_threshold=0.035,
    ).fit_predict(z),
    index=z.index,
)
# Add back ubiquitous and nohit genes as clusters.
clust2 = pd.concat(
    [
        clust2,
        pd.Series(-1, index=drop_ubiq_genes_list),
        pd.Series(-2, index=drop_nohit_genes_list),
    ]
)
clust2_palette = lib.plot.construct_ordered_palette(clust2.unique())

sns.clustermap(
    x.loc[variable_genes],
    row_colors=x.loc[variable_genes].index.to_series().map(clust2).map(clust2_palette),
    col_linkage=strain_geno_linkage,
    col_colors=_col_colors,
    metric="cosine",
)

In [ ]:
clust2_sizes = clust2.value_counts()

clust2_sizes_meta = (
    clust2.value_counts().to_frame("tally")
    # .groupby('clust')
    # .count()
    # .sort_values(ascending=False)
    .assign(
        top5_cc=gene_x_cog_category_matrix.groupby(clust2)
        .sum()
        .rename(int)
        .astype(int)
        .apply(lambda x: x.sort_values(ascending=False).head(5).index.to_list(), axis=1)
    )
)

clust2_sizes_meta.head(10)

In [ ]:
_clust = clust2_sizes_meta.index[9]

_gene_list = idxwhere(clust2 == _clust)
x = strain_gene_uhgg.loc[_gene_list]
y = strain_gene_uhgg_depth.loc[_gene_list]
_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)
_row_colors = (
    gene_annotations.reindex(_gene_list)
    .centroid_99_length.map(np.log10)
    .map(lambda x: x / 4)
    .map(mpl.cm.viridis)
)
_col_linkage = strain_geno_linkage
_row_linkage = sp.cluster.hierarchy.linkage(
    x, method="average", metric="cosine", optimal_ordering=True
)  # TODO

sns.clustermap(
    x,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    xticklabels=0,
    yticklabels=0,
)
sns.clustermap(
    y,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    norm=mpl.colors.PowerNorm(1 / 2, vmin=0, vmax=2),
    xticklabels=0,
    yticklabels=0,
)

print(
    gene_x_cog_category_matrix.reindex(_gene_list)
    .sum()
    .sort_values(ascending=False)[lambda x: x > 0]
    .to_frame("tally")
    .join(cog_category_description)
)
gene_annotations.reindex(_gene_list)

### Found using Ref only

In [ ]:
x = strain_gene_uhgg
y = x[ref_list]

drop_ubiq_genes_list = idxwhere(y.mean(1) == 1)
drop_nohit_genes_list = idxwhere(y.mean(1) == 0)
z = y.drop(drop_ubiq_genes_list + drop_nohit_genes_list)

_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)

clust3 = pd.Series(
    AgglomerativeClustering(
        n_clusters=None,
        metric="correlation",
        linkage="average",
        distance_threshold=0.035,
    ).fit_predict(z),
    index=z.index,
)
# Add back ubiquitous and nohit genes as clusters.
clust3 = pd.concat(
    [
        clust3,
        pd.Series(-1, index=drop_ubiq_genes_list),
        pd.Series(-2, index=drop_nohit_genes_list),
    ]
)
clust3_palette = lib.plot.construct_ordered_palette(clust3.unique())

sns.clustermap(
    x.loc[variable_genes],
    row_colors=x.loc[variable_genes].index.to_series().map(clust3).map(clust3_palette),
    col_linkage=strain_geno_linkage,
    col_colors=_col_colors,
    metric="cosine",
)

In [ ]:
clust3_sizes = clust3.value_counts()

clust3_sizes_meta = (
    clust3.value_counts().to_frame("tally")
    # .groupby('clust')
    # .count()
    # .sort_values(ascending=False)
    .assign(
        top5_cc=gene_x_cog_category_matrix.groupby(clust3)
        .sum()
        .rename(int)
        .astype(int)
        .apply(lambda x: x.sort_values(ascending=False).head(5).index.to_list(), axis=1)
    )
)
clust3_sizes_meta.head(10)

In [ ]:
_clust = clust3_sizes_meta.head(10).index[-1]

_gene_list = idxwhere(clust3 == _clust)
x = strain_gene_uhgg.loc[_gene_list]
y = strain_gene_uhgg_depth.loc[_gene_list]
_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)
_row_colors = (
    gene_annotations.reindex(_gene_list)
    .centroid_99_length.map(np.log10)
    .map(lambda x: x / 4)
    .map(mpl.cm.viridis)
)
_col_linkage = strain_geno_linkage
_row_linkage = sp.cluster.hierarchy.linkage(
    x, method="average", metric="cosine", optimal_ordering=True
)  # TODO

sns.clustermap(
    x,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    xticklabels=0,
    yticklabels=0,
)
sns.clustermap(
    y,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    norm=mpl.colors.PowerNorm(1 / 2, vmin=0, vmax=2),
    xticklabels=0,
    yticklabels=0,
)

print(
    gene_x_cog_category_matrix.reindex(_gene_list)
    .sum()
    .sort_values(ascending=False)[lambda x: x > 0]
    .to_frame("tally")
    .join(cog_category_description)
)
gene_annotations.reindex(_gene_list)

### Comparison Among Clusterings

In [ ]:
_gene_list = idxwhere((gene_prevalence >= 0) & (gene_prevalence <= 1))

print(adjusted_mutual_info_score(clust1[_gene_list], clust3[_gene_list]))
print(adjusted_mutual_info_score(clust1[_gene_list], clust2[_gene_list]))
print(adjusted_mutual_info_score(clust2[_gene_list], clust3[_gene_list]))

In [ ]:
_gene_list = idxwhere((gene_prevalence >= 0.05) & (gene_prevalence <= 0.95))

print(adjusted_mutual_info_score(clust1[_gene_list], clust3[_gene_list]))
print(adjusted_mutual_info_score(clust1[_gene_list], clust2[_gene_list]))
print(adjusted_mutual_info_score(clust2[_gene_list], clust3[_gene_list]))

In [ ]:
_gene_list = idxwhere((gene_prevalence >= 0) & (gene_prevalence <= 1))

print(adjusted_rand_score(clust1[_gene_list], clust3[_gene_list]))
print(adjusted_rand_score(clust1[_gene_list], clust2[_gene_list]))
print(adjusted_rand_score(clust2[_gene_list], clust3[_gene_list]))

In [ ]:
_gene_list = idxwhere((gene_prevalence >= 0.05) & (gene_prevalence <= 0.95))

print(adjusted_rand_score(clust1[_gene_list], clust3[_gene_list]))
print(adjusted_rand_score(clust1[_gene_list], clust2[_gene_list]))
print(adjusted_rand_score(clust2[_gene_list], clust3[_gene_list]))

In [ ]:
from sklearn.metrics.cluster import contingency_matrix

_inspect_clust = clust2
_against_clust = clust3

contingency = pd.DataFrame(
    contingency_matrix(_against_clust, _inspect_clust),
    index=sorted(_against_clust.unique()),
    columns=sorted(_inspect_clust.unique()),
)
spgc_gene_clust_meta = (
        pd.DataFrame(
        dict(
            clust_size=contingency.sum(),
            largest_overlap_size=contingency.max(),
            largest_overlap_clust=contingency.idxmax(),
            prevalence_spgc=(
                strain_gene_uhgg[spgc_list].groupby(_inspect_clust).mean()
            ).mean(1),
            prevalence_ref=(
                strain_gene_uhgg[ref_list].groupby(_inspect_clust).mean()
            ).mean(1),
        )
    )
    .assign(
        interest_score=lambda x: x.prevalence_ref * np.log(x.prevalence_ref) * x.prevalence_spgc * np.log(x.prevalence_spgc) * np.log(x.clust_size),
    )
    .sort_values('interest_score', ascending=False)
)
spgc_gene_clust_meta.head(10)

In [ ]:
_clust = spgc_gene_clust_meta.index[0]

_gene_list = idxwhere(clust2 == _clust)
x = strain_gene_uhgg.loc[_gene_list]
y = strain_gene_uhgg_depth.loc[_gene_list]
_col_colors = pd.DataFrame(
    dict(
        genome_type=genome_type.map(genome_type_palette),
    )
)
_row_colors = (
    gene_annotations.reindex(_gene_list)
    .centroid_99_length.map(np.log10)
    .map(lambda x: x / 4)
    .map(mpl.cm.viridis)
)
_col_linkage = strain_geno_linkage
_row_linkage = sp.cluster.hierarchy.linkage(
    x, method="average", metric="cosine", optimal_ordering=True
)  # TODO

sns.clustermap(
    x,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    xticklabels=0,
    yticklabels=0,
)
sns.clustermap(
    y,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    norm=mpl.colors.PowerNorm(1 / 2, vmin=0, vmax=2),
    xticklabels=0,
    yticklabels=0,
)

print(
    gene_x_cog_category_matrix.reindex(_gene_list)
    .sum()
    .sort_values(ascending=False)[lambda x: x > 0]
    .to_frame("tally")
    .join(cog_category_description)
)
gene_annotations.reindex(_gene_list)

## SPGC-MWAS

In [ ]:
_row_linkage = gene_uhgg_linkage
_gene_data = strain_gene_uhgg

u = (
    sfacts_fit.community.sel(strain=spgc_list)
    .to_pandas()
    .groupby(mgen_meta.subject_id)
    .mean()
    > 0.2
)[lambda x: x.any(1)]
v = _gene_data[u.columns]
subject_strain_gene_content = (u @ v.T).T > 0
subject_strain_gene_prevalence = subject_strain_gene_content.mean(1)

subject_ibd_diagnosis = subject.loc[subject_strain_gene_content.columns].ibd_diagnosis

_col_colors = subject_ibd_diagnosis.replace({"CD": "tab:green", "UC": "tab:blue", "nonIBD": "lightgrey"})
row_order = sp.cluster.hierarchy.to_tree(_row_linkage).pre_order(lambda x: x.id)
sns.clustermap(subject_strain_gene_content.iloc[row_order], col_colors=_col_colors, row_cluster=False)

In [ ]:
mwas_results = []
for _gene in tqdm(strain_gene_uhgg.index):
    contingency_table0 = (
        pd.DataFrame(dict(diagnosis=subject_ibd_diagnosis, gene=subject_strain_gene_content.loc[_gene]))
        .value_counts()
        .unstack('diagnosis')
        .assign(IBD=lambda x: x.CD + x.UC)
    )
    for ibd_comparison, (groupA, groupB) in dict(cd=('nonIBD', 'CD'), uc=('nonIBD', 'UC'), ibd=('nonIBD', 'IBD')).items():
        contingency_table1 = (
            contingency_table0
            .reindex(index=[True, False], columns=[groupA, groupB])
            .fillna(0)
        )
        _test = sp.stats.fisher_exact(contingency_table1)
        contingency_table_pc = contingency_table1 + 1
        oddsratio_pc = (
            contingency_table_pc.loc[True, groupB] / contingency_table_pc.loc[False, groupB]
        ) / (
            contingency_table_pc.loc[True, groupA]
            / contingency_table_pc.loc[False, groupA]
        )
        mwas_results.append((ibd_comparison, _gene, *_test, oddsratio_pc))

mwas_results = (
    pd.DataFrame(
        mwas_results,
        columns=["ibd_comparison", "gene", "statistic", "pvalue", "oddsratio_pc"],
    )
    # .join(
    #     gene_annotations[
    #         [
    #             "Description",
    #             "COG_category",
    #             "eggNOG_OGs",
    #             "centroid_99_length",
    #             "score",
    #             "Preferred_name",
    #             "KEGG_ko",
    #             "PFAMs",
    #         ]
    #     ],
    #     on="gene",
    # )
    .sort_values("pvalue")
)

mwas_results.head(10)

In [ ]:
bins = np.logspace(-5, 0, num=20)
# bins = np.linspace(0, 1, num=51)
for ibd_comparison in mwas_results.ibd_comparison.unique():
    plt.hist(
        mwas_results[lambda x: x.ibd_comparison == ibd_comparison].pvalue,
        label=ibd_comparison,
        alpha=0.5,
        bins=bins,
    )

plt.plot(bins[1:], (bins[1:] - bins[:-1]) * mwas_results.shape[0] / 3)
plt.legend()

plt.xscale("log")
plt.yscale("log")

In [ ]:
_gene = mwas_results.gene.iloc[0]
print(_gene)
pd.DataFrame(dict(diagnosis=subject_ibd_diagnosis, gene=subject_strain_gene_content.loc[_gene])).value_counts().unstack().fillna(0)

In [ ]:
_gene_list = mwas_results.head(10).gene
x = strain_gene_uhgg.loc[_gene_list]
y = strain_gene_uhgg_depth.loc[_gene_list]
_col_colors = genome_type.map(genome_type_palette)
_row_colors = (
    gene_annotations.reindex(_gene_list)
    .centroid_99_length.map(np.log10)
    .map(lambda x: x / 4)
    .map(mpl.cm.viridis)
)
_col_linkage = strain_geno_linkage
_row_linkage = sp.cluster.hierarchy.linkage(
    x, method="average", metric="cosine", optimal_ordering=True
)  # TODO

sns.clustermap(
    x,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    xticklabels=0,
    yticklabels=0,
)
sns.clustermap(
    y,
    figsize=(10, 5),
    row_linkage=_row_linkage,
    col_linkage=_col_linkage,
    col_colors=_col_colors,
    norm=mpl.colors.PowerNorm(1 / 2, vmin=0, vmax=2),
    xticklabels=0,
    yticklabels=0,
)

print(
    gene_x_cog_category_matrix.reindex(_gene_list)
    .sum()
    .sort_values(ascending=False)[lambda x: x > 0]
    .to_frame("tally")
    .join(cog_category_description)
)
gene_annotations.reindex(_gene_list)

In [ ]:
_gene_data = strain_gene_uhgg.loc[_gene_list]

_subject_x_strain = (
    sfacts_fit.community.sel(strain=spgc_list)
    .to_pandas()
    .groupby(mgen_meta.subject_id)
    .mean()
    > 0.2
)[lambda x: x.any(1)]
_strain_x_gene = _gene_data[u.columns]
_subject_x_gene = (_subject_x_strain @ _strain_x_gene.T).T > 0

_col_colors = subject.loc[_subject_x_gene.columns].ibd_diagnosis.replace(
    {"CD": "tab:green", "UC": "tab:blue", "nonIBD": "lightgrey"}
)
sns.clustermap(_subject_x_gene, col_colors=_col_colors, row_cluster=False)

In [ ]:
_gene_data = subject_uhgg_depth_ratio.T.loc[_gene_list].applymap(
    np.nan_to_num, nan=0, posinf=100, neginf=-100
)

_col_colors = pd.DataFrame(
    dict(
        ibd_diagnosis=subject.ibd_diagnosis.replace(
            {"CD": "tab:green", "UC": "tab:blue", "nonIBD": "lightgrey"}
        ),
        species_depth=subject_mean_species_depth.map(
            lambda x: mpl.cm.viridis(x / subject_mean_species_depth.max())
        ),
    )
)
sns.clustermap(
    _gene_data + 1e-5,
    col_colors=_col_colors,
    norm=mpl.colors.SymLogNorm(1e-3, vmin=1e-2, vmax=1e2),
    metric="cosine",
    row_cluster=False,
)

In [ ]:
mean_subject_depth = subject_uhgg_depth_ratio[_gene]

d = subject[["ibd_diagnosis"]].join(mean_subject_depth).dropna()
for ibd_diagnosis in d.ibd_diagnosis.unique():
    plt.hist(
        d.loc[d.ibd_diagnosis == ibd_diagnosis, _gene] + 1e-4,
        bins=np.logspace(-4, 3),
        alpha=0.5,
        label=ibd_diagnosis,
    )

thresh = 1e-1
plt.axvline(thresh, lw=1, linestyle="--", color="k")
plt.xscale("log")
# plt.yscale('log')
plt.legend()

d.assign(gene=lambda x: x[_gene] > thresh)[
    ["ibd_diagnosis", "gene"]
].value_counts().unstack()

## Write Statistics

In [ ]:
gene_stats = (
    pd.DataFrame(dict(
        nlength=gene_annotations['centroid_99_length'],
        eggnog=gene_annotations['eggNOG_OGs'],
        description=gene_annotations['Description'],
        gene_name=gene_annotations['Preferred_name'],
        ko=gene_annotations['KEGG_ko'],
        pfam=gene_annotations['PFAMs'],
        prevalence_spgc=spgc_gene_uhgg_prevalence,
        prevalence_ref=ref_gene_uhgg_prevalence,
        prevalence_mwas_subject=subject_strain_gene_prevalence,
        phylogenetic_r_spgc=phylogenetic_signal_spgc.r_statistic,
        phylogenetic_r_ref=phylogenetic_signal_ref.r_statistic,
        coclust_label_spgc=clust1,
        coclust_label_both=clust2,
        coclust_label_ref=clust3,
    ))
    .join(mwas_results.set_index(['gene', 'ibd_comparison']).pvalue.unstack().rename(columns=lambda x: x + '_mwas_pvalue'))
    .join(mwas_results.set_index(['gene', 'ibd_comparison']).oddsratio_pc.unstack().rename(columns=lambda x: x + '_mwas_oddsratio_pc'))
    .join(gene_x_cog_category_matrix.rename(columns=lambda x: 'cog_category_' + x))
    .rename_axis(index='gene_id')
)

gene_stats.to_csv(gene_stats_outpath, sep='\t')
gene_stats

In [ ]:
spgc_strain_stats = (
    pd.DataFrame(dict(
        nearest_ref_geno=genome_distance_comparison[lambda x: x.genome_type.isin(['SPGC'])]['min_geno_ref'],
        nearest_ref_geno_diss=genome_distance_comparison[lambda x: x.genome_type.isin(['SPGC'])]['geno_dist'],
        nearest_ref_gene_diss=genome_distance_comparison[lambda x: x.genome_type.isin(['SPGC'])]['gene_dist'],
        nearest_ref_gene_raw_diss=genome_distance_comparison_raw[lambda x: x.genome_type.isin(['SPGC'])]['gene_dist'],
    ))
    .join(spgc_meta.loc[spgc_list])
    .rename_axis(index='strain')
)

spgc_strain_stats.to_csv(spgc_strain_stats_outpath, sep='\t')
spgc_strain_stats

In [ ]:
ref_strain_stats = (
    pd.DataFrame(
        dict(
            nearest_ref_geno=genome_distance_comparison[
                lambda x: x.genome_type.isin(["Isolate", "MAG"])
            ]["min_geno_ref"],
            nearest_ref_geno_diss=genome_distance_comparison[
                lambda x: x.genome_type.isin(["Isolate", "MAG"])
            ]["geno_dist"],
            nearest_ref_gene_diss=genome_distance_comparison[
                lambda x: x.genome_type.isin(["Isolate", "MAG"])
            ]["gene_dist"],
            nearest_ref_gene_raw_diss=genome_distance_comparison_raw[
                lambda x: x.genome_type.isin(["Isolate", "MAG"])
            ]["gene_dist"],
            genome_type=genome_distance_comparison[
                lambda x: x.genome_type.isin(["Isolate", "MAG"])
            ]["genome_type"],
        )
    )
    .join(
        reference_meta[
            [
                "Length",
                "N_contigs",
                "N50",
                "GC_content",
                "Completeness",
                "Contamination",
            ]
        ]
    )
    .rename_axis(index="strain")
)

ref_strain_stats.to_csv(ref_strain_stats_outpath, sep='\t')
ref_strain_stats